In [1]:
import tweepy
from tweepy import OAuthHandler
import time
from datetime import datetime, timezone
from time import gmtime, strftime
import json
import pandas as pd
import os

In [2]:
# Obtain list of Twitter profile IDs
df1 = pd.read_excel('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/MasterDataExtract.xlsx', dtype = object)
df1 = df1[(df1.Plattform == 'Twitter') & (df1.KeySearchType == 'ProfileID')]
TWProfileList = df1.KeySearch.values.tolist()
TWAliasList = df1.Alias.values.tolist()

In [3]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_PrepareOutputIntoDF.ipynb

In [4]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_TextPreparation.ipynb

In [5]:
# Initialize Twitter API

# Twitter credentials for the app
consumer_key = 'consumer_key' # API Key
consumer_secret = 'consumer_secret' # API Secret
access_key = 'access_key' # Access token
access_secret = 'access_secret' # Access secret token

# Pass consumer key and secret
auth = OAuthHandler(consumer_key, consumer_secret)

# Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
apiTW = tweepy.API(auth)

In [6]:
# Function to extract from API, tweets per profileID and return selected fields in dataframe form

def TweetPerProfile(userID01, alias01):

    tweetrepliesDF0 = pd.DataFrame()
    tweetsList = []
    j = 1

    for tweet in tweepy.Cursor(apiTW.search, q = 'to:'+userID01, result_type = 'recent',
                               timeout = 999999, tweet_mode = 'extended', exclude_replies = False,
                               include_rts = True).items(1000):
        tweetsList.append(tweet)
        
        if j % 100 == 0:
            time.sleep(2)
            j = j + 1
        
    
    for twr in tweetsList:
        twDict = json.loads(json.dumps(twr._json))
        tweetreplyoutput = {'Output1': {'User': alias01, 
                                        'tweetContent': {'created_at': twDict.get('created_at'), 
                                                         'id_str': twDict.get('id_str'),
                                                         'in_reply_to_status_id_str': twDict.get('in_reply_to_status_id_str'),
                                                         'retweet_count': twDict.get('retweet_count'),
                                                         'favorite_count': twDict.get('favorite_count'),
                                                         'lang': twDict.get('lang'),
                                                         'full_text': TextCleansing(twDict.get('full_text')),
                                                         
                                                         'userDict': {'id_str': twDict.get('user').get('id_str'), 
                                                                      'name': TextCleansing(twDict.get('user').get('name')), 
                                                                      'location': TextCleansing(twDict.get('user').get('location')), 
                                                                      'followers_count': twDict.get('user').get('followers_count'),
                                                                      'friends_count': twDict.get('user').get('friends_count'),
                                                                      'created_at': twDict.get('user').get('created_at')}}},
                            'Key1': userID01, 'Key2': twDict.get('id_str')}

        tweetrepliesDF0 = tweetrepliesDF0.append(tweetreplyoutput, ignore_index = True, sort = False)
    
    return tweetrepliesDF0

In [7]:
i = 0
tweetDF = pd.DataFrame()
tweetIDLAll = []
for TWProfile in TWProfileList:
    print('***', TWProfile, '\nstart\t', datetime.now())
    tweetoutputDF = TweetPerProfile(TWProfile, TWAliasList[i])
    tweetDF = tweetDF.append(tweetoutputDF, ignore_index=True, sort = False)
    i = i + 1
    print('end\t', datetime.now())
    time.sleep(60)

*** JMilei 
start	 2021-08-02 16:15:38.743954
end	 2021-08-02 16:16:41.874276
*** jlespert 
start	 2021-08-02 16:17:41.887633
end	 2021-08-02 16:18:40.961734
*** LibertarioCABA 
start	 2021-08-02 16:19:40.969272
end	 2021-08-02 16:19:51.552765


In [8]:
tweetDF = PrepareOutputDF('K12', tweetDF, 'Automatic', 'TweetsPerUser-Twitter')

In [9]:
DF = pd.read_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index_col = False, encoding = 'ISO-8859-1')
DF = DF.append(tweetDF, ignore_index = True)
DF.to_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index = False)